# manuscriptFiles

> This module has commands to handle the manuscript file structure and relates the annotation application to files saved in the manuscripts folder

In [1]:
#| default_exp manuscriptFiles

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
import os
import io
import cv2
import numpy as np

## createManuscriptDirectory

The ```createManuscriptDirectory``` function creates a new manuscript directory from scratch, including:
- a manuscript image folder
- an annotation states folder
- a metadata file
- a folder for exported transcriptions
- a folder for imported transcriptions

This function relies on two methods: ```directoryNameClean``` and ```dictToList```. These are simple calls for simple purposes, but just complicated enough to warrant their own functions for future optimizations.

In [4]:
#| export
def createManuscriptDirectory(metadata:dict):
    # This function creates a directory and metadata file for a new manuscript and returns the new manuscript root directory
    
    # Establishing the baseDirectory the web app is running in
    baseDirectory = os.getcwd()
    os.chdir('manuscripts')
    allManuscriptsDirectory = os.getcwd()

    # Creating the root directory for a new manuscript
    title = directoryNameClean(metadata['Work'])
    manuscriptDirectory = os.path.join(allManuscriptsDirectory,title)
    os.mkdir(manuscriptDirectory)
    
    # Creating directories
    imagesDirectory = os.path.join(manuscriptDirectory,'images')
    os.mkdir(imagesDirectory)
    statesDirectory = os.path.join(manuscriptDirectory,'states')
    os.mkdir(statesDirectory)
    exportTranscriptDirectory = os.path.join(manuscriptDirectory,'exportTranscripts')
    os.mkdir(exportTranscriptDirectory)
    importTranscriptDirectory = os.path.join(manuscriptDirectory,'importTranscripts')
    os.mkdir(importTranscriptDirectory)
    
    # Creating metadata file as config file
    os.chdir(manuscriptDirectory)
    f = open(title + '.cfg', 'w')
    
    # Writes relevant metadata to file
    printable = dictToList(metadata)
    for data in printable:
        f.write(data + '\n')
    
    os.chdir(baseDirectory)
    
    return manuscriptDirectory

In [87]:
# Testing on a real world example
Stav53 = {
    'Work': 'Stavronikita Monastery Greek handwritten document Collection no.53',
    'Author': '',
    'Language': 'Greek',
    'Country': 'Greece',
    'City': 'Mount Athos',
    'Institution': 'Stavronikita Monastery',
    'Centuries': '14th Century'
}

createManuscriptDirectory(Stav53)

'/home/dc/glyptodon/manuscripts/stvrnktmnstrygrkcllctnn.53'

## dictToList

The ```dictToList``` function creates a printable list out of a dictionary that keeps keys separate from the items stored to them and creates a list that can be printed to a file, specifically the manuscript ```title.cfg``` file.

In [6]:
#| export
def dictToList(thisdict:dict):
    # This function turns a dictionary into a list of printable strings
    keys = []
    for key in thisdict.keys():
        keys.append(key)

    values = []
    for value in thisdict.values():
        values.append(value)
    
    printable = []
    for i in range(len(keys)):
        printable.append(str(keys[i]) + ':' + str(values[i]))
    
    return printable

In [7]:
# Testing dictToList

thisdict = {
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}

printable = dictToList(thisdict)

for i in range(len(printable)):
    print(printable[i])

brand:Ford
model:Mustang
year:1964


## directoryNameClean

The ```directoryNameClean``` function removes the characters from a string that are not allowed characters for a directory name and shortens the name. It is called in other manuscript file methods and is indirectly used to generate a folder key for the select manuscripts. It cuts down the length of the title to no more than 26 characters to avoid creating excessively long directory titles and surpassing the 260 character limit. While it is unlikely for the path limit to be reached, caution is good. It proceeds in this order:
- Remove words that are not capitalized at the beginning or are not numerical
- Remove illegal characters
- If name length is greater than 26:
  - Remove vowels
  - Remove trailing characters

In [85]:
#| export
def directoryNameClean(string):
    # This breaks a string down into individual words and
    words = string.split()
    upperWords = []
    for word in words:
        if word[0].isupper():
            upperWords.append(word)
        elif word.isalpha() == False:
            upperWords.append(word)
    
    string = ''
    for word in upperWords:
        string = string + word
    
    # This loop removes any of the illegal characters for directories
    illegalChars = ['\\','#','%','&','{','}','<','>','*','?','/',' ','$','!',"'",'"',':','@','+','`','|','=']
    for char in illegalChars:
        removalCount = 0
        numChars = len(string)
        for i in range(len(string)):
            if string[i] == char:
                string = string[:i] + string[i+1:]
                removalCount = removalCount + 1

            if numChars - removalCount - 1 == i:
                break
    
    # This loop removes any vowels in an overly long string
    vowels = ['a','e','i','o','u']
    if len(string) > 26:
        for char in vowels:
            removalCount = 0
            numChars = len(string)
            for i in range(len(string)):
                if string[i] == char:
                    while string[i] == char:
                        string = string[:i] + string[i+1:]
                        removalCount = removalCount + 1
                
                if numChars - removalCount - 1 == i:
                    break
    
    if len(string) > 26:
        string = string[0:26]
    
    return string.lower()

In [86]:
# Testing the function on a real world example
newstring = 'Stavronikita Monastery Greek handwritten document Collection no.53'

print(directoryNameClean(newstring))

stvrnktmnstrygrkcllctnn.53


In [24]:
# Testing the directoryNameClean function. ```re.sub``` didn't work for whatever reason, something about nothing being at 0, despite that not being an expected behavior. Difficulty iterating over multiple chars? Also didn't take backslashes at all.

illegalChars = ['\\','#','%','&','{','}','<','>','*','?','/',' ','$','!',"'",'"',':','@','+','`','|','=']
newstring=''

for char in illegalChars:
    newstring = newstring + char + 'aaa'
    
print(newstring)

print(directoryNameClean(newstring))

\aaa#aaa%aaa&aaa{aaa}aaa<aaa>aaa*aaa?aaa/aaa aaa$aaa!aaa'aaa"aaa:aaa@aaa+aaa`aaa|aaa=aaa
a


## saveImage

The ```saveImage``` function takes image content from the upload widget value dictionary and saves them as files to the relevant manuscripts folder. Borrows code from [Ulrich Stern's answer](https://stackoverflow.com/a/54162776/1628638) and iterates over the elements of the dictionary with that code. It takes the arguments:
- ```files```: a dictionary with image data for several images keyed in memory
- ```targetDirectory```: a path to some manuscript directory for image saving

Passed into the ```files``` argument **MUST** be a ```<FileUpload widget>.value``` key from an uploader with image data. This contains the memory locations for the images.

In [10]:
#| export
def saveImages(files:dict, targetDirectory):
    # This function saves content from memory into storage using the keys in the passed files dict (from a FileUpload widget)
    # This 
    baseDirectory = os.getcwd()
    os.chdir(os.path.join(targetDirectory, 'images'))
    
    # Not sure what exactly this does, this is what I borrowed
    for i in range(len(files)):
        # I think this takes the content from the memory marker and reads it into Python readable code
        img_stream = io.BytesIO(files[i]['content'])
        # I have no clue why Numpy is needed here (probably some ungodly matrices and formats), but this turns the data into a cv2 readable image
        img = cv2.imdecode(np.frombuffer(img_stream.read(), np.uint8), 1)
        # This writes an image to some target directory
        cv2.imwrite(files[i]['name'], img)
        
    os.chdir(baseDirectory)

In [11]:
# testing for saveImages
import ipywidgets as widgets

uploader = widgets.FileUpload(
    accept = '',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple = True,  # True to accept multiple files upload else False
    description = 'Upload Manuscript Images',
    layout = widgets.Layout(height='auto', width='auto')
)
uploader

FileUpload(value=(), description='Upload Manuscript Images', layout=Layout(height='auto', width='auto'), multi…

In [12]:
target = os.path.join(os.path.join(os.getcwd(), 'manuscripts'), 'elements')

saveImages(uploader.value, target)

FileNotFoundError: [Errno 2] No such file or directory: '/home/dc/glyptodon/manuscripts/manuscripts/elements/images'

In [13]:
uploader.value

()

## currentManuscripts

We also want a function that can search through the current directories in manuscripts and find all the manuscripts in the directory to return the metadata of the manuscripts. To that end, we have the ```currentManuscripts``` function.

In [14]:
#| export

def currentManuscripts():
    # If this is run on any computer, it will have a unique file structure. This implementation works with that file structure.
    baseDirectory = os.getcwd()
    manuscriptDirectory = os.path.join(baseDirectory,'manuscripts')
    
    # This is necessary to keep directories accessible. Without os.path.join, we can't keep a full directory name and access files inside specific directories
    directories = []
    for path in os.listdir(manuscriptDirectory):
        directories.append(os.path.join(manuscriptDirectory,path))
    
    # This is necessary to store metadata from .cfg files
    manuscriptMetadata = []
    
    # This is necessary to search each directory in the manuscripts folder
    for directory in directories:
        # This looks through each file in a given directory
        for file in os.listdir(directory):
            # This opens config files and reads metadata from them
            if file.endswith('.cfg'):
                fileDirectory = os.path.join(directory,file)
                f = open(fileDirectory, 'r')
                metadata = {}

                for line in f:
                    key, value = line.split(':')
                    metadata[key] = value[:-1]

                manuscriptMetadata.append([directory, metadata])
    
    os.chdir(baseDirectory)
    return manuscriptMetadata

In [15]:
test = currentManuscripts()

print(len(test))

for metadata in test:
    print(metadata[1])

FileNotFoundError: [Errno 2] No such file or directory: '/home/dc/glyptodon/manuscripts/manuscripts'

In [16]:
for metadata in test:
    print(type(metadata[0]))

NameError: name 'test' is not defined

## zipManuscripts [in progress]

If we ever want to export transcriptions from inside this document, having an automatic zip function would be incredibly useful. To that end, here is some testing on a file zipping function.

In [17]:
# Reading in file directories from zipTest folder to zip them

baseDirectory = os.getcwd()
zipDirectory = 'zipTest'

files = []
paths = []
for path in os.listdir(zipDirectory):
    if path == 'images':
        paths.append(os.path.join(zipDirectory, path))
    elif path == 'exportTranscripts':
        paths.append(os.path.join(zipDirectory, path))

for path in paths:
    for file in os.listdir(path):
        files.append(os.path.join(path, file))
        
print(files)
# not sure if this is needed, but keeping it for best practice
os.chdir(baseDirectory)

FileNotFoundError: [Errno 2] No such file or directory: 'zipTest'

In [ ]:
# Trying different zip strategies
import zipfile

current = os.getcwd()

# base folder size is 8641 KB

# This iterates over a list of files
for file in files:
    # This writes a file to a particular zip folder
    zipfile.ZipFile(file = 'zipTest.zip', mode = 'a', compression = zipfile.ZIP_DEFLATED,).write(file)

# no compresslevel corresponds to 8156 KB

In [ ]:
## Testing compression levels
for file in files:
    # This writes a file to a particular zip folder
    zipfile.ZipFile(file = 'zipTest1.zip', mode = 'a', compression = zipfile.ZIP_DEFLATED, compresslevel = 1).write(file)

# result: 8153 KB
    
for file in files:
    # This writes a file to a particular zip folder
    zipfile.ZipFile(file = 'zipTest2.zip', mode = 'a', compression = zipfile.ZIP_DEFLATED, compresslevel = 2).write(file)

# result: 8151 KB
    
for file in files:
    # This writes a file to a particular zip folder
    zipfile.ZipFile(file = 'zipTest3.zip', mode = 'a', compression = zipfile.ZIP_DEFLATED, compresslevel = 3).write(file)

# result: 8150 KB

# results for levels 4-9: 8156 KB

for file in files:
    # This writes a file to a particular zip folder
    zipfile.ZipFile(file = 'zipTest4.zip', mode = 'a', compression = zipfile.ZIP_DEFLATED, compresslevel = 4).write(file)
    
for file in files:
    # This writes a file to a particular zip folder
    zipfile.ZipFile(file = 'zipTest5.zip', mode = 'a', compression = zipfile.ZIP_DEFLATED, compresslevel = 5).write(file)

for file in files:
    # This writes a file to a particular zip folder
    zipfile.ZipFile(file = 'zipTest6.zip', mode = 'a', compression = zipfile.ZIP_DEFLATED, compresslevel = 6).write(file)

for file in files:
    # This writes a file to a particular zip folder
    zipfile.ZipFile(file = 'zipTest7.zip', mode = 'a', compression = zipfile.ZIP_DEFLATED, compresslevel = 7).write(file)
    
for file in files:
    # This writes a file to a particular zip folder
    zipfile.ZipFile(file = 'zipTest8.zip', mode = 'a', compression = zipfile.ZIP_DEFLATED, compresslevel = 8).write(file)

for file in files:
    # This writes a file to a particular zip folder
    zipfile.ZipFile(file = 'zipTest9.zip', mode = 'a', compression = zipfile.ZIP_DEFLATED, compresslevel = 9).write(file)

## updateMetadata

This function is designed to update the metadata in a file given a dictionary of information being passed into it and a path to the relevant directory. This dictionary should be based on calls from ```Text``` widgets in the Information stage. This was designed using [this answer](https://stackoverflow.com/questions/3964681/find-all-files-in-a-directory-with-extension-txt-in-python/3964690#3964690) from stackoverflow.

In [2]:
#| export
def updateMetadata(directory, information):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.cfg'):
                f = open(file, 'w')
                
                printable = dictToList(information)
                for data in printable:
                    f.write(data + '\n')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()